## Coursera Capstone Project - Week 3 - Segmenting and Clustering Toronto Neighborhoods
### Question 1 -- scraping the Wikipedia webpage and creating the dataset

In [1]:
import pandas as pd

In [2]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [3]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df.shape

(180, 3)

In [5]:
import numpy as np

# replace "Not assigned" to NaN
df.replace("Not assigned", np.nan, inplace = True)
df.head(5)


,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
df.dropna(subset=["Borough"], axis=0, inplace=True)

In [7]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df.shape

(103, 3)

### Question 2 -- Adding latitude and longitude

In [11]:
df_coords=pd.read_csv("http://cocl.us/Geospatial_data")
df.shape

(103, 3)

In [12]:
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_coords_sorted = df_coords.sort_values(by=['Postal Code'])
df_coords_sorted.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_coords_sorted.reset_index(drop=True, inplace=True)
df_coords_sorted.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_sorted = df.sort_values(by=['Postal Code'])

In [16]:
df_sorted.head()

,Postal Code,Borough,Neighbourhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [17]:
df_sorted.reset_index(drop=True, inplace=True)
df_sorted.shape

(103, 3)

In [18]:
df_sorted.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
df_sorted['Latitude'] = df_coords_sorted['Latitude']

In [20]:
df_sorted.head()

,Postal Code,Borough,Neighbourhood,Latitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573
3,M1G,Scarborough,Woburn,43.770992
4,M1H,Scarborough,Cedarbrae,43.773136


In [21]:
df_sorted['Longitude'] = df_coords_sorted['Longitude']

In [22]:
df_sorted.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Question 3 -- Segmenting and clustering Toronto neighborhoods



In [23]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json # library to handle JSON files
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium==0.5
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 79 kB 9.5 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=8b6b982806a6eb36b594aa2150f7005e14c5b1002eb628413128d107f3e70566
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


In [24]:
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.820952, -79.3052626.


In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_sorted['Latitude'], df_sorted['Longitude'], df_sorted['Borough'], df_sorted['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [26]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(df_sorted['Borough'].unique()),
        df_sorted.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighbourhoods.


In [27]:
downtownToronto_data = df_sorted[df_sorted['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtownToronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [28]:
downtownToronto_data.shape

(19, 5)

In [29]:
CLIENT_ID = 'PLVCWWLH4OLCZWPQ0XSVAOIHIBUF3ARXXT530SUHAJFKO5OJ' # your Foursquare ID
CLIENT_SECRET = '4CFUBKQ5LBVMZPZLHX1AVKAP2ANWGKO3POYMPZNARTAEUENC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PLVCWWLH4OLCZWPQ0XSVAOIHIBUF3ARXXT530SUHAJFKO5OJ
CLIENT_SECRET:4CFUBKQ5LBVMZPZLHX1AVKAP2ANWGKO3POYMPZNARTAEUENC


In [30]:
downtownToronto_data.loc[0, 'Neighbourhood']

'Rosedale'

In [31]:
neighbourhood_latitude = downtownToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = downtownToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = downtownToronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [32]:
#GET request from Foursquare API for URL
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PLVCWWLH4OLCZWPQ0XSVAOIHIBUF3ARXXT530SUHAJFKO5OJ&client_secret=4CFUBKQ5LBVMZPZLHX1AVKAP2ANWGKO3POYMPZNARTAEUENC&ll=43.6795626,-79.37752940000001&v=20180605&radius=500&limit=100'

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60104d5d2dd20635be23062f'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aff2d47f964a520743522e3',
       'name': 'Rosedale Park',
       'location': {'address': '38 Scholfield Ave.',
        'crossStreet': 'at Edgar Ave.',
        'lat': 43.68232820227814,
        'lng': -79.37893434347683,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68232820227814,
          'lng': -79.37893434347683}],
        'distance': 32

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


#### Exploring neighbourhoods in downtown Toronto

In [37]:
#Exploring neighbourhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
downtownToronto_venues = getNearbyVenues(names=downtownToronto_data['Neighbourhood'],
                                  latitudes=downtownToronto_data['Latitude'],
                                  longitudes=downtownToronto_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [39]:
print(downtownToronto_venues.shape)
downtownToronto_venues.head()

(1219, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [40]:
downtownToronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,60,60,60,60,60,60
Christie,17,17,17,17,17,17
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [41]:
print('There are {} uniques categories.'.format(len(downtownToronto_venues['Venue Category'].unique())))

There are 208 uniques categories.


#### Analyzing each neighbourhood

In [42]:
#Analyzing each neighbourhood
# one hot encoding
downtownToronto_onehot = pd.get_dummies(downtownToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtownToronto_onehot['Neighbourhood'] = downtownToronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtownToronto_onehot.columns[-1]] + list(downtownToronto_onehot.columns[:-1])
downtownToronto_onehot = downtownToronto_onehot[fixed_columns]

downtownToronto_onehot.head()



,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
downtownToronto_onehot.shape

(1219, 209)

In [44]:
#calculating the mean of frequency of occurrence of each venue
downtownToronto_grouped = downtownToronto_onehot.groupby('Neighbourhood').mean().reset_index()
downtownToronto_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.00,0.00,0.018182,0.00000,0.00000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,...,0.000000,0.0000,0.00,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
2,Central Bay Street,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.00,0.00,0.016667,0.00000,0.00000,0.016667,0.000000,0.016667
3,Christie,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.00,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.0125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.000000,...,0.012500,0.0125,0.00,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.025000
5,"Commerce Court, Victoria Hotel",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.0000,0.00,0.00,0.020000,0.00000,0.00000,0.010000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.010000,0.0000,0.00,0.01,0.010000,0.00000,0.00000,0.010000,0.000000,0.000000
7,"Garden District, Ryerson",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.020000,0.0000,0.00,0.00,0.000000,0.01000,0.01000,0.010000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.0000,0.00,0.01,0.010000,0.00000,0.00000,0.010000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.00,0.00,0.049180,0.00000,0.04918,0.016393,0.000000,0.000000


In [45]:
downtownToronto_grouped.shape

(19, 209)

In [46]:
#printing out top 5 venues of each neighbourhood (postal code)
num_top_venues = 5

for hood in downtownToronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtownToronto_grouped[downtownToronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2      Restaurant  0.04
3  Farmers Market  0.04
4     Cheese Shop  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3     Boat or Ferry  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.05
2      Sandwich Place  0.05
3                Café  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3     Restaurant  0.06
4           Bank  0.06


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.09
1      Sushi Restaurant  0.06
2   Japanese Restaurant  0.06
3            Restaurant  0.04
4  Fast Food Restaurant  0.0

In [47]:
#making a pandas dataframe out of thie venues / neighbourhood information
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtownToronto_grouped['Neighbourhood']

for ind in np.arange(downtownToronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtownToronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Seafood Restaurant,Bakery,Cheese Shop,Farmers Market,Park,Irish Pub
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Bar,Coffee Shop
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Office,Miscellaneous Shop,Modern European Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Bank,Baby Store,Athletics & Sports,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Restaurant,Gay Bar,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel


#### Clustering neighbourhoods

In [49]:
#clustering the neighbourhoods
# set number of clusters
kclusters = 5

downtownToronto_grouped_clustering = downtownToronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtownToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4], dtype=int32)

In [50]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtownToronto_merged = downtownToronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtownToronto_merged = downtownToronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtownToronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Playground,Trail,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Café,Italian Restaurant,Bakery,Restaurant,Pizza Place,Pub,Chinese Restaurant,Deli / Bodega,Beer Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Restaurant,Gay Bar,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Cosmetics Shop,Restaurant,Brewery
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hotel,Ramen Restaurant


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtownToronto_merged['Latitude'], downtownToronto_merged['Longitude'], downtownToronto_merged['Neighbourhood'], downtownToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examining Clusters

In [59]:
#Examining the individual clusters
#Cluster 0 -- Airport
downtownToronto_merged.loc[downtownToronto_merged['Cluster Labels'] == 0, downtownToronto_merged.columns[[2] + list(range(5, downtownToronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, King and Spadina, Railway Lands, Har...",0,Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Sculpture Garden,Boat or Ferry,Rental Car Location,Bar,Coffee Shop


In [60]:
#Cluster 1 -- Coffee shops, restaurants, shopping, gyms
downtownToronto_merged.loc[downtownToronto_merged['Cluster Labels'] == 1, downtownToronto_merged.columns[[2] + list(range(5, downtownToronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"St. James Town, Cabbagetown",1,Coffee Shop,Café,Italian Restaurant,Bakery,Restaurant,Pizza Place,Pub,Chinese Restaurant,Deli / Bodega,Beer Store
2,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Restaurant,Gay Bar,Pizza Place,Men's Store,Mediterranean Restaurant,Hotel
3,"Regent Park, Harbourfront",1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Cosmetics Shop,Restaurant,Brewery
4,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Hotel,Ramen Restaurant
5,St. James Town,1,Coffee Shop,Café,Gastropub,American Restaurant,Cocktail Bar,Seafood Restaurant,Restaurant,Park,Clothing Store,Moroccan Restaurant
6,Berczy Park,1,Coffee Shop,Cocktail Bar,Beer Bar,Restaurant,Seafood Restaurant,Bakery,Cheese Shop,Farmers Market,Park,Irish Pub
7,Central Bay Street,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Office,Miscellaneous Shop,Modern European Restaurant
8,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Gym,Bakery,Hotel,Deli / Bodega,Thai Restaurant,Clothing Store,Burrito Place
9,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Italian Restaurant,Brewery,Restaurant,Fried Chicken Joint,Bank
10,"Toronto Dominion Centre, Design Exchange",1,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant


In [57]:
#Cluster 2 -- Grocery store, cafe, park, candy store, shopping, restaurant = more geared toward kids and families
downtownToronto_merged.loc[downtownToronto_merged['Cluster Labels'] == 2, downtownToronto_merged.columns[[1] + list(range(5, downtownToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,2,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Bank,Baby Store,Athletics & Sports,Coffee Shop


In [61]:
#Cluster 3 -- Eating places, restaurants, park, playground
downtownToronto_merged.loc[downtownToronto_merged['Cluster Labels'] == 3, downtownToronto_merged.columns[[2] + list(range(5, downtownToronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,3,Park,Playground,Trail,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [63]:
#Cluster 4 -- Cafes, bars, restaurants eating places, college
downtownToronto_merged.loc[downtownToronto_merged['Cluster Labels'] == 4, downtownToronto_merged.columns[[2] + list(range(5, downtownToronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"University of Toronto, Harbord",4,Café,Bookstore,Bar,Japanese Restaurant,Bakery,Sandwich Place,Italian Restaurant,Beer Bar,Beer Store,College Gym
13,"Kensington Market, Chinatown, Grange Park",4,Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Gaming Cafe,Farmers Market,Dessert Shop,Caribbean Restaurant,Grocery Store


#### End of project